In [1]:
import numpy as np
import cv2
from rembg import remove

Denoise step.

In [17]:
output = cv2.imread('./test/Castoria_002.jpg')
sav_pth = './test/Castoria_002_deno.jpg'
# output = output[1732-1142:-1, :] #* Cropping step.
h = 10
#* Size in pixels of the template patch that is used to compute weights. Should be odd. Recommended value 7 pixels
templateWindowSize = 7
#* Size in pixels of the window that is used to compute weighted average for given pixel. Should be odd. Affect performance linearly: greater searchWindowsSize - greater denoising time. Recommended value 21 pixels
searchWindowSize = 21
output_ = cv2.fastNlMeansDenoisingColored(output, None, h, h, templateWindowSize, searchWindowSize)
cv2.imwrite(sav_pth, output_)

cv2.imshow('output_1', output_)

cv2.waitKey(0)
cv2.destroyAllWindows()

Cutout step.

In [ ]:
input_path = './test/Castoria_002_deno.jpg'
output_path = './test/Castoria_002_deno_cutout.jpg'

with open(input_path, 'rb') as i:
    with open(output_path, 'wb') as o:
        input = i.read()
        output = remove(input)
        o.write(output)

Histogram equilization + matching step.

In [82]:
pth_raw = './test/Castoria_002_deno_cutout.jpg'
pth_ref = './test/reference_raw.png'
pth_out = './test/hist_match.jpg'

img_raw = cv2.imread(pth_raw)

img_ref =  cv2.imread(pth_ref)
img_raw_ = img_raw.copy()

def hist_match(img_raw, referensi):

    img_raw_shape = img_raw.shape
    
    img_raw = img_raw.flatten()
    referensi = referensi.flatten()

    o_values, bin_idx, o_counts = np.unique(img_raw, return_inverse=True,return_counts=True)
    b_values, b_counts = np.unique(referensi, return_counts=True)

    o_quantiles = np.cumsum(o_counts).astype(np.float64) #* Cumulative sum
    o_quantiles /= o_quantiles[-1] #* Normalization
    b_quantiles = np.cumsum(b_counts).astype(np.float64)
    b_quantiles /= b_quantiles[-1]

    interp_t_values = np.interp(o_quantiles, b_quantiles, b_values)

    #* [bin_idx] expand the unique array into the original shape.
    return interp_t_values[bin_idx].reshape(img_raw_shape)

for i in range(3):
    img_raw[:,:,i] = hist_match(img_raw[:,:,i], img_ref[:,:,i])

cv2.imwrite(pth_out, img_ref)

True

Dual-denoise.

In [56]:
output = cv2.imread('./test/hist_match.jpg')
sav_pth = './test/hist_match_deno.png'
h = 3
#* Size in pixels of the template patch that is used to compute weights. Should be odd. Recommended value 7 pixels
templateWindowSize = 7
#* Size in pixels of the window that is used to compute weighted average for given pixel. Should be odd. Affect performance linearly: greater searchWindowsSize - greater denoising time. Recommended value 21 pixels
searchWindowSize = 21
output_ = cv2.fastNlMeansDenoisingColored(output, None, h, h, templateWindowSize, searchWindowSize)
cv2.imwrite(sav_pth, output_)

cv2.imshow('output_1', output_)

cv2.waitKey(0)
cv2.destroyAllWindows()

Adjust alpha.

In [81]:
mask = cv2.imread('./test/Castoria_002_deno_cutout.jpg')
mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
# Apply a threshold to the grayscale image
_, threshold = cv2.threshold(mask, 0, 255, cv2.THRESH_BINARY)
t_x, t_y = np.where(threshold==0)
t_ind = np.concatenate((t_x.reshape(-1,1), t_y.reshape(-1,1)), axis=1) #* Return transparent coordinates.

new = cv2.imread('./test/hist_match_deno.png')
new = cv2.cvtColor(new, cv2.COLOR_BGR2RGBA)
sav_new = './test/hist_match_deno_cutout.png'
alpha = new[:, :, 3]
alpha[t_x, t_y] = 0
new[:, :, 3] = alpha
new = cv2.cvtColor(new, cv2.COLOR_RGBA2BGRA)

cv2.imwrite(sav_new, new)

cv2.imshow('inpaint', new)

cv2.waitKey(0)
cv2.destroyAllWindows()